El siguiente sistema de ecuaciones diferenciales explica la poblacion de contagio de cualquier epidemia en el tiempo t:

$$\frac{dS_t}{dt}=b_1-(b_2+\beta_I I_t+\beta_A A_t+\beta_D D_t) S_t$$
$$\frac{dE_t}{dt}=(\beta_I I_t+\beta_A A_t+\beta_D D_t) S_t-(b_2+\gamma)E_t$$
$$\frac{dI_t}{dt}=\gamma p E_t-(b_2+\alpha+\tau_0)I_t+\mu I_t-\xi$$
$$\frac{dA_t}{dt}=\gamma (1-p) E_t-(b_2+\tau_0)A_t$$
$$\frac{dD_t}{dt}=b_2(I_t+A_t)+\alpha I_t-\mu D_t$$
$$\frac{dR_t}{dt}=\tau_0(I_t+A_t)-(b_2+\eta)R_t$$

Donde $S_t$, $I_t$, $R_t$ y $D_t$ corresponden a las proporcion de la gente que es susceptible a contagiarse, los infestados, los recuperados y los mertos respectivamente para el tiempo $t$, y las contantes $\beta$, $\mu$ y $\nu$, corresponden a las tasas de infeccion, de recuperacion y de mortandad respectivamente.
Como la poblacion total $P_t$ es invariante en el tiempo, esto es:

$$\frac{dP_t}{dt}=0$$

Y como la poblacion esta dada por la suma de los 6 grupos a ver, es decir:

$$P_t=S_t+I_t+R_t+D_t+E_t+A_t$$

Por lo que la suma del sistema de ecuaciones diferenciales tiene verificar:

$$\frac{dS_t}{dt}+\frac{dI_t}{dt}+\frac{dR_t}{dt}+\frac{dD_t}{dt}+\frac{dE_t}{dt}+\frac{dA_t}{dt}=0$$

De esta manera el sistema evolutivo de la epidemia seria de la siguiente forma:

In [ ]:
#@title Condiciones Iniciales
Poblacion_total = 10000000  #@param {type: "number"}
Tiempo_total =   200#@param {type: "number"}
Investados_iniciales =   5#@param {type: "number"}
beta = 0.08 #@param {type:"slider", min:0, max:1, step:0.01}
mu = 0.06  #@param {type: "slider", min: 0, max: 1, step:0.01}
nu = 0.1  #@param {type: "slider", min: 0, max: 1, step:0.01}
porcentaje_tratamiento = 0.25  #@param {type: "slider", min: 0, max: 1, step:0.01}

In [ ]:
#@title
import numpy as np
S0=(Poblacion_total-Investados_iniciales)/Poblacion_total
I0=Investados_iniciales/Poblacion_total
R0=0
D0=0
ED0=np.array([[S0],[I0],[R0],[D0]])
ED=ED0
#metodo de Euler
def Fmatriz(beta,mu,nu,E):
  matrix=np.array([[-beta*E[1][0],-beta*E[0][0],0,0],[beta*E[1][0],beta*E[0][0]-mu-nu*E[3][0],0,-nu*(E[1][0]+1)],[0,mu,0,0],[0,nu*E[3][0],0,nu*(E[1][0]+1)]])
  return np.matmul(matriz,E) 
#runge-kutta 4
for i in range(Tiempo_total):
  K1=Fmatriz(beta,mu,nu,ED0)
  K2=Fmatriz(beta,mu,nu,ED0+K1/2)
  K3=Fmatriz(beta,mu,nu,ED0+K2/2)
  K4=Fmatriz(beta,mu,nu,ED0+K3)
  ED0=ED0+(K1+K2+K3+K4)/6
  ED=np.concatenate([ED,ED0],axis=1)
St=ED[0,:]
It=ED[1,:]
Rt=ED[2,:]
Dt=ED[3,:]



from scipy.integrate import ode

y0, t0 = [1.0j, 2.0], 0

def f(t, y, arg1):
    return [1j*arg1*y[0] + y[1], -arg1*y[1]**2]
def jac(t, y, arg1):
    return [[1j*arg1, 1], [0, -arg1*2*y[1]]]
r = ode(f, jac).set_integrator('zvode', method='bdf', with_jacobian=True)
r.set_initial_value(y0, t0).set_f_params(2.0).set_jac_params(2.0)
t1 = 10
dt = 1
while r.successful() and r.t < t1:
    r.integrate(r.t+dt)
    print("%g %g" % (r.t, r.y))

In [ ]:
#@title
import plotly.graph_objects as go
import numpy as np
x = np.arange(Tiempo_total)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x,y=St,mode='lines',name="Susectible a infectarse"))

fig.add_trace(go.Scatter(x=x,y=It,mode='lines',name="Infectados"))

fig.add_trace(go.Scatter(x=x,y=Rt,mode='lines',name="Recuperados"))

fig.add_trace(go.Scatter(x=x,y=Dt,mode='lines',name="Muertos"))

fig.add_trace(go.Scatter(x=x,y=np.repeat(porcentaje_tratamiento,x.shape),mode='lines',name="Camas disponibles"))

fig.update_layout(title='Curva de comportamiento de infestados de alguna Epidemia',
                   xaxis_title='Tiempo [dias]',
                   yaxis_title='Poblacion [%]')

fig.show()

In [ ]:
#@title Relacion entre Suceptibles,Infestados, Recuperados y Muertos
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter3d(
    x=St,
    y=It,
    z=Rt,
    mode='markers',
    marker=dict(
        size=10,
        color=Dt,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        colorbar=dict(
            title="Muertos[%]"
        )
    )
)])
fig.update_layout(scene = dict(
                    xaxis_title='Suceptibles [%]',
                   yaxis_title='Infestados [%]',
                  zaxis_title='Recuperados [%]'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

fig.show()

In [ ]:
#@title Casos Reales Covid-19
#Datos de https://www.worldometers.info/coronavirus/
import pandas as pd
cov=pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv")
cov["dateRep"]=pd.to_datetime(cov["dateRep"],format="%d/%m/%Y")
countries=cov.countriesAndTerritories.drop_duplicates().str.replace("_"," ").tolist()

In [ ]:
#@title
import ipywidgets as widgets
from IPython.display import display
country=widgets.Dropdown(
    options=countries,
    value='Chile',
    description='Countries:',
    disabled=False,
)
display(country)

In [ ]:
import urllib3
import numpy as np
from bs4 import BeautifulSoup
url = "https://www.worldometers.info/world-population/"+country.value.replace(" ","_").lower()+"-population/"
http = urllib3.PoolManager()
response = http.request('GET',url)
soup = BeautifulSoup(response.data)
s=pd.Series(soup.find_all("tbody"))
s=pd.Series(s[0].find_all("strong"))
N=int(str(s[0]).split(">")[1].split("<")[0].replace(",",""))
cov=cov[cov.countriesAndTerritories==country.value.replace(" ","_")].sort_values(["dateRep"]).reset_index(drop=True)
st=np.where((cov.cases>0)|(cov.deaths>0))[0][0]
cov=cov.iloc[st:].reset_index(drop=True)
mu=(158705-151312)/(723390)
cov["nu"]=cov.deaths/cov.cases
cov["mu"]=mu
cov["beta"]=cov.cases/(cov.cases.cumsum())
cov=cov[["dateRep","cases","deaths","nu","beta","mu"]]
prob=np.log1p(cov[["nu","beta","mu"]])-np.log1p(1-cov[["nu","beta","mu"]])

In [ ]:
#@title Condiciones Iniciales
Tiempo_a_estimar =   200#@param {type: "number"}
porcentaje_tratamiento = 0.25  #@param {type: "slider", min: 0, max: 1, step:0.01}

In [ ]:
#@title Correlacion entre las tasas
(cov[["nu","beta","mu"]].corr()*100).round(2)

In [ ]:
#@title Covarianza de la matriz de los logit de las tasas
prob[["nu","beta","mu"]].cov()

Como no existe correlacion entre las tasas $\nu$, $\beta$ y $\mu$, además de que las covarianza de mu es relativamente 0 versus las otras variables, podemos hacer un modelo estocastico de prediccion de contagio, para las tasas $\nu$ y $\beta$, dejando a $\mu$ como una variable constante, ya que aun no existe cura del coronavirus, por lo que solo depende de la tasa natural de recuperacion.

In [ ]:
#@title
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
xv, yv = np.meshgrid(np.arange(10),np.arange(10))
varmax=GridSearchCV(sm.tsa.VARMAX(prob[["nu","beta"]]),
                    param_grid={"order":zip(xv.ravel(),yv.ravel())})
mod = sm.tsa.VARMAX(prob[["nu","beta"]], order=varmax.best_param_["order"])
res = mod.fit(maxiter=1000, disp=False)
Pred=res.predict(start=prob.shape[0], end=prob.shape[0]+Tiempo_a_estimar)
Pred1=1/(1+np.exp(-Pred))
Pred1["mu"]=cov.mu.iloc[0]
A=pd.concat([cov[["nu","beta","mu"]],Pred1],axis=0).reset_index(drop=True)

In [ ]:
#@title
import numpy as np
#metodo de Euler
def matriz(beta,mu,nu,S0,I0,D0):
  return np.array([[-beta*I0,-beta*S0,0,0],[beta*I0,beta*S0-mu-nu,0,0],[0,mu,0,0],[0,nu,0,0]])
for i in range(A.shape[0]):
  if i>cov.shape[0]-1:
    B=matriz(A.beta[i],A.mu[i],A.nu[i],ED0[0][0],ED0[1][0],ED0[3][0])
    ED1=np.matmul(B,ED0)+ED0
    ED0=ED1
  else:
    ED0=np.array([[1-cov.loc[:i+1,["cases","deaths"]].sum().sum()/N,cov.loc[:i+1,"cases"].sum()/N,0,cov.loc[:i+1,"deaths"].sum()/N]]).reshape(4,1)
  if i==0:
    ED=ED0
  else:
    ED=np.concatenate([ED,ED0],axis=1)
St=ED[0,:]
It=ED[1,:]
Rt=ED[2,:]
Dt=ED[3,:]
x=pd.date_range(start=cov.dateRep.iloc[0],end=cov.dateRep.iloc[0]+pd.to_timedelta(A.shape[0], unit='D'),freq="D")

In [ ]:
#@title
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()
fig.add_trace(go.Scatter(x=x,y=St*N,mode='lines',name="Susectible a infectarse"))

fig.add_trace(go.Scatter(x=x,y=It*N,mode='lines',name="Infectados"))

fig.add_trace(go.Scatter(x=x,y=Rt*N,mode='lines',name="Recuperados"))

fig.add_trace(go.Scatter(x=x,y=Dt*N,mode='lines',name="Muertos"))

fig.add_trace(go.Scatter(x=x,y=np.repeat(porcentaje_tratamiento,x.shape)*N,mode='lines',name="Camas disponibles"))

fig.update_layout(title='Curva de comportamiento de infestados de alguna Epidemia en '+country.value,
                   xaxis_title='Tiempo [dias]',
                   yaxis_title='Poblacion[hab]')

fig.show()

In [ ]:
#@title Relacion entre Suceptibles,Infestados, Recuperados y Muertos
import plotly.graph_objects as go


fig = go.Figure(data=[go.Scatter3d(
    x=St,
    y=It,
    z=Rt,
    mode='markers',
    marker=dict(
        size=10,
        color=Dt,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        colorbar=dict(
            title="Muertos[%]"
        )
    )
)])
fig.update_layout(scene = dict(
                    xaxis_title='Suceptibles [%]',
                   yaxis_title='Infestados [%]',
                  zaxis_title='Recuperados [%]'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

fig.show()

Este pequeño ejemplo fue realizado por:

>Ronald Humberto Castillo Capino.

>Ingeniero Matemático y Científico de Datos.

>$\int_{\mathbb{N}} {g(e^{m}) \forall t1 \mathbb{C}0}$

>Fono: +56 9 5913 6403

> ron.h.castillo@gmail.com

>[![GITHUB](https://cdn4.iconfinder.com/data/icons/social-media-2097/94/linkedin-128.png)](https://www.linkedin.com/in/ronald-humberto-castillo-capino-b99a5413/)

>[![GITHUB](http://st-hadoop.cs.umn.edu/sites/st-hadoop.dl.umn.edu/files/styles/panopoly_image_original/public/media/github-logo.png?itok=OcFuVGNk)](https://github.com/R0N4L2)